https://docs.aws.amazon.com/sagemaker/latest/dg/mxnet-example1.html

https://github.com/aws/sagemaker-python-sdk

http://sagemaker.readthedocs.io/en/latest/

In [46]:
from sagemaker.mxnet import MXNet
from sagemaker import get_execution_role

role = get_execution_role()

mnist_estimator = MXNet(entry_point='part2_sm_mnist.py',
                        py_version='py3',
                        role=role,
                        train_instance_count=1, 
                        train_instance_type='ml.p2.xlarge')

This creates a Sagemaker Job, which you can find at https://console.aws.amazon.com/sagemaker/home?region=us-east-1#/jobs

In [47]:
mnist_estimator.fit(inputs={"train":"s3://jakechenawspublic/sample_data/mnist/train/",
                            "test":"s3://jakechenawspublic/sample_data/mnist/test/"})

INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-148356438281
INFO:sagemaker:Creating training-job with name: sagemaker-mxnet-py2-gpu-2018-01-09-04-01-01-141


....................................................................................
executing startup script (first run)
2018-01-09 04:07:53,515 INFO - root - running container entrypoint
2018-01-09 04:07:53,516 INFO - root - starting train task
2018-01-09 04:07:55,236 INFO - mxnet_container.train - MXNetTrainingEnvironment: {'enable_cloudwatch_metrics': False, 'available_gpus': 1, 'channels': {u'test': {u'TrainingInputMode': u'File', u'RecordWrapperType': u'None', u'S3DistributionType': u'FullyReplicated'}, u'train': {u'TrainingInputMode': u'File', u'RecordWrapperType': u'None', u'S3DistributionType': u'FullyReplicated'}}, '_ps_verbose': 0, 'resource_config': {u'current_host': u'algo-1', u'hosts': [u'algo-1']}, 'user_script_name': u'part2_sm_mnist.py', 'input_config_dir': '/opt/ml/input/config', 'channel_dirs': {u'test': u'/opt/ml/input/data/test', u'train': u'/opt/ml/input/data/train'}, 'code_dir': '/opt/ml/code', 'output_data_dir': '/opt/ml/output/data/', 'output_dir': '/opt/ml/out

2018-01-09 04:08:43,615 INFO - root - Epoch[6] Batch [200]#011Speed: 24729.65 samples/sec#011accuracy=0.989900
2018-01-09 04:08:44,018 INFO - root - Epoch[6] Batch [300]#011Speed: 24835.77 samples/sec#011accuracy=0.989800
2018-01-09 04:08:44,418 INFO - root - Epoch[6] Batch [400]#011Speed: 25001.19 samples/sec#011accuracy=0.992000
2018-01-09 04:08:44,823 INFO - root - Epoch[6] Batch [500]#011Speed: 24720.10 samples/sec#011accuracy=0.991000
2018-01-09 04:08:45,180 INFO - root - Epoch[6] Train-accuracy=0.989663
2018-01-09 04:08:45,181 INFO - root - Epoch[6] Time cost=2.373
2018-01-09 04:08:45,204 INFO - root - Epoch[6] Validation-accuracy=0.988000
2018-01-09 04:08:45,609 INFO - root - Epoch[7] Batch [100]#011Speed: 24830.96 samples/sec#011accuracy=0.990000
2018-01-09 04:08:46,012 INFO - root - Epoch[7] Batch [200]#011Speed: 24860.85 samples/sec#011accuracy=0.991700
2018-01-09 04:08:46,414 INFO - root - Epoch[7] Batch [300]#011Speed: 24895.13 samples/sec#011accuracy=0.990800
2018-01-09 04

This fit() function automatically downloads the file(s). Recall from part2 that these file locations are in the 'channel_input_dirs' argument going into the training script's train() method.

In [ ]:
predictor = mnist_estimator.deploy(instance_type="ml.m4.xlarge",
                                   initial_instance_count=1)

INFO:sagemaker:Creating model with name: sagemaker-mxnet-py2-gpu-2018-01-09-04-01-01-141
INFO:sagemaker:Creating endpoint with name sagemaker-mxnet-py2-gpu-2018-01-09-04-01-01-141


--------------------------

In [54]:
predictor.predict()

ValueError: Unable to handle input format: 